이번 과제는 완성하지 못해서 기존 keras로 짜여진 character based rnn 모델을 가져와서 테스트, 분석해봤습니다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!ls "/content/drive/My Drive"

'3D Sketch (3).pptx'	    Object_detection_webcam_fix.py
 arpen.apk		   '세미나 ppt 책자.zip'
'Colab Notebooks'	    rhyme.txt
 Desktop.zip		    source.txt
 hand_finger_detect.apk     testset_png.zip
 handrecognition_model.h5   testtmp.zip
 HW1.docx		   'Vuforia-Brush-master-fix(client)_final.zip'
 leap_database.zip	   '앵커 기반 드론 측위에 대한 부품 명세서.xlsx'
 object_detection.py.zip   '특허 관련.zip'


In [3]:
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# save tokens to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()
 
# load text
raw_text = load_doc('/content/drive/My Drive/source.txt')
print(raw_text)    # 소스코드의 원본
 
# clean
tokens = raw_text.split()
raw_text = ' '.join(tokens)
 
# organize into sequences of characters
length = 10
sequences = list()
for i in range(length, len(raw_text)):   # 시퀀스들의 전체 갯수는 전체 텍스트 길이 - 하나의 시퀀스 길이가 됩니다.
	# select sequence of tokens
	seq = raw_text[i-length:i+1]           # 개별 시퀀스 각각을 좌에서 우로 sweeping 하며 순차적으로 sequences에 추가합니다.
	# store
	sequences.append(seq)
print('Total Sequences: %d' % len(sequences))
 
# save sequences to file
out_filename = 'char_sequences.txt'
save_doc(sequences, out_filename)    # 시퀀스들의 리스트를 저장해둡니다.

void MainWindow::ConvertFromPFILE(const string& fileIn,
    const string& clientId,
    const string& redirectUrl,
    const string& clientEmail) 
{
// add trusted certificates using HttpHelpers of RMS and Auth SDKs
addCertificates();

// create shared in/out streams
auto inFile = make_shared<ifstream>(
fileIn, ios_base::in | ios_base::binary);

if (!inFile->is_open()) {
 AddLog("ERROR: Failed to open ", fileIn.c_str());
return;
}

string fileOut;

// generate output filename
auto pos = fileIn.find_last_of('.');

if (pos != string::npos) {
 fileOut = fileIn.substr(0, pos);
}

 // create streams
auto outFile = make_shared<fstream>(
fileOut, ios_base::in | ios_base::out | ios_base::trunc | ios_base::binary);

if (!outFile->is_open()) {
 AddLog("ERROR: Failed to open ", fileOut.c_str());
 return;
  }

try
{
// create authentication context
AuthCallback auth(clientId, redirectUrl);

// process conversion
auto pfs = PFileConverter::ConvertFromPFile(
  clientEmail,
  inFile,
  outFile,
  aut

In [4]:
print(sequences)

['void MainWi', 'oid MainWin', 'id MainWind', 'd MainWindo', ' MainWindow', 'MainWindow:', 'ainWindow::', 'inWindow::C', 'nWindow::Co', 'Window::Con', 'indow::Conv', 'ndow::Conve', 'dow::Conver', 'ow::Convert', 'w::ConvertF', '::ConvertFr', ':ConvertFro', 'ConvertFrom', 'onvertFromP', 'nvertFromPF', 'vertFromPFI', 'ertFromPFIL', 'rtFromPFILE', 'tFromPFILE(', 'FromPFILE(c', 'romPFILE(co', 'omPFILE(con', 'mPFILE(cons', 'PFILE(const', 'FILE(const ', 'ILE(const s', 'LE(const st', 'E(const str', '(const stri', 'const strin', 'onst string', 'nst string&', 'st string& ', 't string& f', ' string& fi', 'string& fil', 'tring& file', 'ring& fileI', 'ing& fileIn', 'ng& fileIn,', 'g& fileIn, ', '& fileIn, c', ' fileIn, co', 'fileIn, con', 'ileIn, cons', 'leIn, const', 'eIn, const ', 'In, const s', 'n, const st', ', const str', ' const stri', 'const strin', 'onst string', 'nst string&', 'st string& ', 't string& c', ' string& cl', 'string& cli', 'tring& clie', 'ring& clien', 'ing& client', 'ng& clie

In [5]:
from numpy import array
from pickle import dump
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load
in_filename = 'char_sequences.txt'
raw_text = load_doc(in_filename)
lines = raw_text.split('\n')   # 각 시퀀스들을 line 단위로 분할하여 lines에 저장합니다.

# integer encode sequences of characters
chars = sorted(list(set(raw_text)))       # set을 통해 전체 텍스트에 속해있는 character들의 unique 리스트를 만들고 정렬합니다.
mapping = dict((c, i) for i, c in enumerate(chars))   # enumerate를 통해 각 인덱스마다 번호와 char의 튜플을 구하여 딕셔너리를 만듭니다.
sequences = list()
for line in lines:
	# integer encode line
	encoded_seq = [mapping[char] for char in line]
	# store
	sequences.append(encoded_seq)    # 모든 코드를 mapping 딕셔너리를 통해 정수값으로 매핑합니다.

Using TensorFlow backend.


In [6]:
print(chars)
print(mapping)
print(sequences)

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '/', '0', ':', ';', '<', '=', '>', 'A', 'C', 'D', 'E', 'F', 'H', 'I', 'K', 'L', 'M', 'O', 'P', 'R', 'S', 'U', 'W', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', '{', '|', '}']
{'\n': 0, ' ': 1, '!': 2, '"': 3, '&': 4, "'": 5, '(': 6, ')': 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, ':': 13, ';': 14, '<': 15, '=': 16, '>': 17, 'A': 18, 'C': 19, 'D': 20, 'E': 21, 'F': 22, 'H': 23, 'I': 24, 'K': 25, 'L': 26, 'M': 27, 'O': 28, 'P': 29, 'R': 30, 'S': 31, 'U': 32, 'W': 33, '_': 34, 'a': 35, 'b': 36, 'c': 37, 'd': 38, 'e': 39, 'f': 40, 'g': 41, 'h': 42, 'i': 43, 'k': 44, 'l': 45, 'm': 46, 'n': 47, 'o': 48, 'p': 49, 'r': 50, 's': 51, 't': 52, 'u': 53, 'v': 54, 'w': 55, 'x': 56, 'y': 57, '{': 58, '|': 59, '}': 60}
[[54, 48, 43, 38, 1, 27, 35, 43, 47, 33, 43], [48, 43, 38, 1, 27, 35, 43, 47, 33, 43, 47], [43, 38, 1, 27, 35, 43, 47, 33, 43, 47, 38], [38, 1, 27, 35, 

In [7]:
# vocabulary size
vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)

# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]  # 마지막 글자를 제외한 모든 행렬과  마지막 글자만 가지고 있는 모든 행렬로 나눕니다.
sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = array(sequences)
y = to_categorical(y, num_classes=vocab_size)   # X와 y에 대하여 원핫 인코딩 해줍니다.

Vocabulary Size: 61


In [8]:
print(X)
print(y)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]]]
[[0. 0. 0.

In [0]:
# define model
model = Sequential()
model.add(LSTM(75, input_shape=(X.shape[1], X.shape[2])))   # LSTM 레이어에 X를 input으로 넣어줍니다.
model.add(Dense(vocab_size, activation='softmax'))    # 총 char 들의 사이즈 만큼이 클래스(아웃풋)이 됩니다.
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, epochs=100, verbose=2)

# save the model to file
model.save('model.h5')
# save the mapping
dump(mapping, open('mapping.pkl', 'wb'))

In [5]:
from pickle import load
from keras.models import load_model
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
	in_text = seed_text
	# generate a fixed number of characters
	for _ in range(n_chars):
		# encode the characters as integers
		encoded = [mapping[char] for char in in_text]  # 모든 seed 텍스트를 정수로 매핑
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')  # 동일한 길이로 패딩
		# one hot encode
		encoded = to_categorical(encoded, num_classes=len(mapping))
		encoded = encoded.reshape(-1, encoded.shape[1], encoded.shape[2])
		# predict character
		yhat = model.predict_classes(encoded, verbose=0)   # predict하여 yhat에 저장
		# reverse map integer to character
		out_char = ''
		for char, index in mapping.items():
			if index == yhat:  # 일치하는 인덱스 발견하면
				out_char = char   # char로 변환
				break
		# append to input
		in_text += char  # 결과적으로 in_text는 주어진 seed 문자열에서의 각 char마다 predict한 것들로 채워진 문자열이 된다.
	return in_text

# load the model
model = load_model('model.h5')
# load the mapping
mapping = load(open('mapping.pkl', 'rb'))

# test start of rhyme
print(generate_seq(model, mapping, 10, 'void MainWindow:', 20))   # seed 문자열을 주고 모델이 predict 하게 함
# test mid-line
print(generate_seq(model, mapping, 10, 'generate out', 20))
# test not in original
print(generate_seq(model, mapping, 10, 'hello worl', 20))  # 원본 소스에 존재하지 않았던 문자열은 올바른 예측을 할 수 없음

void MainWindow::ConvertFromPFile( c
generate output filename auto po
hello worlCettrerer, fioste::e


In [6]:
print(generate_seq(model, mapping, 10, 'void MainWindow:', 200))  # 원본 소스에 존재했던 문자열은 정확하게 예측함

void MainWindow::ConvertFromPFile( clientEmail) { // add trusted certificates using HttpHelpers of RMS and Auth SDKs addCertificates using HttpHelpers of RMS and Auth SDKs addCertificates using HttpHelpers of RMS and


원래는 indent까지 고려해야 하는데, 이는 위의 코드에서 split을 통해 개행을 제거하여 tokenize 하는 부분을 수정하면 될 것 같습니다.